In [1]:
%load_ext autoreload
%autoreload 2

In [17]:
import sys
sys.path.append('..')

import os
import numpy as np
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
import multiprocessing
from functools import partial
from PIL import Image
import numbergen as ng
import imagen as ig
from skimage.transform import resize
import time
import cv2
from ssn.random_pattern import random_pattern
import numbergen as ng
import imagen as ig
import random

In [7]:
pattern_generator = random_pattern()

In [9]:
def show(img, title='',cmap=None): 
    plt.figure(figsize=(15,10))
    plt.imshow(img, cmap=cmap)
    plt.title(title)
    plt.show()

In [11]:
testing_ibl_folder = '/home/ysheng/Dataset/ibls/pattern/'
testing_real_ibl_folder = '/home/ysheng/Dataset/ibls/real/'

In [54]:
sample_dimension = 10
num_list = [i for i in range(50)]
size_list = [i * 0.01 for i in range(1,11)]

print(num_list)
print(scale_list)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001, 0.8, 0.9, 1.0, 1.1, 1.2000000000000002, 1.3, 1.4000000000000001, 1.5, 1.6, 1.7000000000000002, 1.8, 1.9000000000000001, 2.0, 2.1, 2.2, 2.3000000000000003, 2.4000000000000004, 2.5, 2.6, 2.7, 2.8000000000000003, 2.9000000000000004, 3.0]


In [49]:
def normalize(ibl, energy=3500):
    total_energy = np.sum(ibl)
    if total_energy < 1e-3:
        print('small energy: ', total_energy)
        return np.zeros((80,512))

    ibl = ibl * energy / total_energy

    return ibl

def to_normalized_ibl(ibl):
    """ input is pattern generated ibl [80x512]
        output is noramlized ibl: [16x32]
    """
    ibl = cv2.resize(ibl, (32, 5))
    ret = np.zeros((16, 32))
    ret[:5,:] = ibl
    return ret

## prepare testing ibls

In [56]:
pattern = pattern_generator.get_pattern()
scale = 3.0
size_list.reverse()
for num in tqdm(num_list):
    for size in size_list:
        seed = random.randint(0,19920208)
        factor = 80/256
        if num == 0:
            ibl = np.zeros((80,512))
        else:
            gs = ig.Composite(operator=np.add,
                              generators=[ig.Gaussian(
                                        size=size*ng.UniformRandom(seed=seed+i+4),
                                        scale=scale*(ng.UniformRandom(seed=seed+i+5)+1e-3),
                                        x=ng.UniformRandom(seed=seed+i+1)-0.5,
                                        y=(ng.UniformRandom(seed=seed+i+2)-0.5)*factor,
                                        aspect_ratio=0.7,
                                        orientation=np.pi*ng.UniformRandom(seed=seed+i+3),
                                        ) for i in range(num)],
                                position=(0, 0), 
                                xdensity=512)
            ibl = normalize(gs()[80:160,:], 3500) 
        title = 'num_{}_size_{}_ibl.png'.format(num, size)
#         show(ibl, title)
        ibl = to_normalized_ibl(ibl)
#         show(ibl, title)
        cv2.imwrite(os.path.join(testing_ibl_folder, title),ibl)

100%|██████████| 50/50 [01:21<00:00,  1.63s/it]


## prepare real testing ibls 

## prepare animation ibls

## output one batch data